# [AHA! Activity Health Analytics](http://casas.wsu.edu/)
[Center for Advanced Studies of Adaptive Systems (CASAS)](http://casas.wsu.edu/)

[Washington State University](https://wsu.edu)

# L5 Physical Activity Change Detection

## Learner Objectives
At the conclusion of this lesson, participants should have an understanding of:
* Physical Activity Change Detection (PACD)
* Common change point detection algorithms

## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Sprint et al., 2016](http://www.sciencedirect.com/science/article/pii/S1532046416300740)

## Physical Activity Change Detection Overview
In the previous lessons, we learned how to download time series data from the Fitbit API and how to visualize the time series data. For the example Fitbit data in [fitbit_example_data.zip](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_example_data.zip), we created an ADM that looked like the following:
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/ADM_example.png" width="700">

Visual inspection of this ADM tells us quite a bit about the activity patterns for this individual. He/she usually sleeps from 1AM-8AM, is fairly sedentary during the week, and goes out for exercise on the weekends from 12PM-5PM. Visual inspection is useful for [exploratory data analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis) (EDA). Beyond the EDA stage of analyzing this dataset, we want to write an algorithm to *quantify* the change between two different windows of time. For example, if we could quantify the daily change from weekday to weekday, or weekend to weekend, we could potentially flag certain daily routines as "normal" or "abnormal", depending on the value of the detected change. 

The previous example is a special case of a general problem called change detection and analysis. In [Sprint et al., 2016](http://www.sciencedirect.com/science/article/pii/S1532046416300740), we address this problem with our Physical Activity Change Detection (PACD) approach. PACD is a framework that determines if a significant change exists between two windows of time series step data (data which indicates the number of steps taken by an individual over a period of time) sampled from a physical activity sensor, such as a Fitbit. PACD is comprised of four main steps:
1. Segments time series data into windows
2. Detects changes between time windows
3. Determines significance of the detected changes
4. Analyzes the nature of the significant changes.

<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/PACD_overview.png" width="800">

Let's go through each one of these steps in more detail.

### Segment Data
Recall the steps data frame we constructed in the previous lesson, [fitbit_example_data_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_example_data_steps_df.csv). The index of this data frame is a minute by minute `DateTimeIndex` and the columns are daily step data. Here is a sample of the format of the data:

|time|10/1/2015|10/2/2015|...|10/21/2015|
|-|-|-|-|-|
|0:00:00|0|0|...|0|
|0:01:00|0|0|...|0|
|...|...|...|...|...|
|23:59:00|0|7|...|0|

To perform segmentation, consider the above data frame example. Let $D = \{x_{1}, x_{2},..., x_{t},..., x_{m}\}$ represent one column of time series step data for one day, where $x_{t}$ is a scalar number of steps taken at time interval $t = 1, 2, ..., m$ and $m$ is the number of equal-sized time intervals in a day. Let $t_{mins}$ denote the number of minutes per time interval, $t$. For the above example, $t_{mins} = 1$ minute and $m = 1440$ minutes / $t_{mins}$ = 1440 intervals; however, if we re-sample the data, say hourly, then $t_{mins} = 60$ minutes and $m = 1440$ minutes / $t_{mins}$ = 24 intervals. $N$ days $D$ (columns) together represents a time series data set $X$ (data frame) of $N$ days, $D_{1}, ..., D_{N}$. Now, let $W$ be a window of $n$ days such that $W \subseteq X$. You can think of $W$ as a slice of $X$ along the date (column) axis.

We can compare windows of data within time series data $X$. These windows may represent consecutive times (e.g., days, weeks, months), a baseline window (e.g., the first week) with each subsequent time window, or overlapping windows. Let $W_{i}$ denote a window starting at day number $i$ of $X$ ($i \geq 1$) such that $W_{i} = X[i: i + n - 1] = \{D_{i}, D_{i+1},..., D_{i+n-1}\}$. Suppose we have two windows of data, $W_{i}$ and $W_{j}$ ($i \leq j$). Windows $W_{i}$ and $W_{j}$ can be formed as subsets of $X$ based on the initial value of $i$ and a parameter $\mathit{offset}$ that determines the initial value of $j$ ($j$ = $i + \mathit{offset}$). 

We categorize choices for window size $n$ into the following descriptors: 
1. Small window ($n$ = 1 day). Suitable for performing day-to-day comparisons (e.g. $D_{i(Monday)}$ compared to $D_{j(Monday)}$, $D_{i(Tuesday)}$ compared to $D_{j(Tuesday)}$, ...) or aggregate day comparisons (e.g. $\hat{W}_{i}$ compared to $\hat{W}_{j}$, $\hat{W}_{i + adv_{i}}$ compared to $\hat{W}_{j + adv_{j}}$, ...).
1. Medium window (2 days $\leq n \leq$ 5 days). Suitable for performing weekday-to-weekday comparisons (e.g. $W_{i}$ compared to $W_{j}$ where $W_{i} = \{D_{i(Monday)}$, $D_{i(Tuesday)}$, $D_{i(Wednesday)}$, $D_{i(Thursday)}$, $D_{i(Friday)}\}$ and $W_{j} = \{D_{j(Monday)}$, $D_{j(Tuesday)}$, $D_{j(Wednesday)}$, $D_{j(Thursday)}$, $D_{j(Friday)}\}$) or weekend-to-weekend comparisons.
1. Large window ($n >$ 5 days). Suitable for performing week-to-week or month-to-month comparisons.

### Detect Changes
For change detection and analysis, a function $F$ computes a change score, $CS = F(W_{i}, W_{j})$ between $W_{i}$ and $W_{j}$.  Two windows can be compared in either baseline or sliding window mode. 
1. Baseline window comparison: the first window is a reference window that occurs at the beginning of the time series ($i$ is initialized to 1 and stays at 1) and is used in each comparison. All subsequent windows are compared to the baseline window ($j$ advances).
1. Sliding window comparison: both windows used for comparison are advanced through the time series data ($i$ is initialized to 1 and $j$ is initialized to $1 + \mathit{offset}$). The windows slide together, staying separated by a number of days specified by $\mathit{offset}$.

<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/PACD_segment_change.png" width="800">

There are several algorithmic options for the window-based changes score function, $F$. Here is a short list of algorithms with references for further reading if you are interested:
1. [Virtual classifier (VC)](https://link.springer.com/chapter/10.1007%2F978-3-540-68125-0_15): computes the accuracy of a binary classifier trained with feature vectors ($F_{i}$ and $F_{j}$) to discriminate between two windows.
1. [Texture-based dissimilarity](http://ieeexplore.ieee.org/document/6189784/?section=abstract): computes the dissimilarity (Euclidean distance) between a vector of image processing-based features (subsets of $F_{i}$ and $F_{j}$) from each window.
1. [Small Window Permutation-based Change Detection (sw-PCAR)](http://www.sciencedirect.com/science/article/pii/S1532046416300740): using hypothesis testing, computes a p-value based on shuffling time intervals between two windows.
1. [Relative Unconstrained Least-Squares Importance Fitting (RuLSIF)](http://www.sciencedirect.com/science/article/pii/S0893608013000270): measures the difference between two samples of data surrounding between two windows using direct density ratio estimation.

Here is a table the describes the different aspects of each algorithm:

|Approach|Window size|Window preprocessing|Change score|Change significance test|
|-|-|-|-|-|
|Virtual classifier|Large|Physical activity features (intra-day and inter-day if window size $>$ 1)|Cross validation prediction accuracy of binary classifier|Hypothesis testing based on prediction accuracy exceeding a threshold|
|Texture-based|Any|Grey-level co-occurrence matrix, texture features|Weighted normalized Euclidean distance|N/A|
|sw-PCAR|Any|$N$ Kullback-Leibler distance permutation vector ($N$ = number of permutations)|Kullback-Leibler distance|Non-parametric outlier detection based on Boxplot analysis| 
|RuLSIF|Any|Hankel matrix|Probability density ratio estimation with Pearson divergence|Threshold learning in supervised applications. N/A for unsupervised applications|

In the subsequent lesson, we will take a closer look at the VC approach.

### Test for Change Significance
Significance testing of change score $CS$ is necessary to interpret change score values. With significance testing, we are answering the question, "Is the magnitude of $CS = F(W_{i}, W_{j})$ significantly large?" 

The VC approach, includes a test of significance to determine if a significant change exists between the two windows, $W_{i}$ and $W_{j}$. We will look at this test in detail in the subsequent lesson. RuLSIF, texture-based, and sw-PCAR do not have associated change significance tests. We have proposed change significance tests that are described in detail in [Sprint et al., 2016](http://www.sciencedirect.com/science/article/pii/S1532046416300740).

<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/PACD_change_significance.png" width="200">

### Change Analysis
If a change significance test concludes a change score is significant, the next step is to determine the source of change. Often this step requires the computation of features that summarize the data and provide a meaningful context for change. For example, the number of daily steps taken is an example of a simple PA feature. The change between daily steps from one window of time to the next can be quantified and used for an explanation of change. 

Several approaches exist to capture change across time in individual metrics. A straightforward method is to compute the percent change for a feature $f$ from a previous window $W_{i}$ to a current window $W_{j}$: $\Delta\%=({f_{W_{j}}-f_{W_{i}}})/f_{W_{i}}$. Statistical approaches such as two sample tests or effect size analyses can also be applied to quantify change; however, in applying repeated statistical tests, the multiple testing problem should be accounted for with a method such as the Bonferroni or Benjamini-Hochberg correction.

One of the advantages of the VC approach over other change point detection algorithms is it includes an explanation of the source of change without reliance on statistical tests. Upon significant change detection, retraining a classifier on the entire dataset and inspecting the model reveals which features are most discriminatory in learning the differences between two windows. Naturally, this approach requires a pre-processing step to extract relevant features from the windowed PA time series data.